In [1]:
!pip install -q transformers sentencepiece

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "tsmatz/mt5_summarize_japanese"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def summarize(text: str, target_chars=120):
    """
    text : 入力テキスト
    target_chars : 「だいたいこのくらいの文字数で書いてほしい」という希望
    """
    # mT5 は「文字数」ではなく「トークン数」で制限される → だいたいでOK
    max_length = int(target_chars * 1.3)  # 少し余裕を持たせる

    prompt = (
        f"次の文章を、だいたい {target_chars} 文字くらいの長さで自然に要約してください。\n"
        f"【文章】{text}\n"
        "【要約】"
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# === 実行部分 ===
text = input("要約したいテキストを貼ってください：\n")
target = int(input("だいたい何文字くらいで要約してほしいですか？："))

print("\n--- 要約結果 ---\n")
result = summarize(text, target_chars=target)
print(result)
print("\n【実際の文字数】：", len(result))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/867 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

要約したいテキストを貼ってください：
もぎたて石手　　（３／８ ＯＡ）　ナレ原稿 スタジオ①  【Ｒ-１】  四国霊場、第５１番札所 石手寺。  きょうのもぎたては 春のかおりが漂う、石手寺の周りを歩きます。   何よりも健康が一番！ こんな時期だからこそ、笑顔になりましょう！  今、家にいるあなた！ 一緒にもぎたてテレビを作りませんか？？   ということで、みなさんから 「私にとっての春」大募集！ お便り待ってまーす   人がおいしい！と思ううまみが 酒造りでは雑味になってしまうんだとか。  杜氏の大山さんは お米をどれだけ削るか、お酒の生産量などを決める酒造りの最高責任者。 酒造りの職人「蔵人」とともに冬の間日本酒を仕込みます。   清酒の仕込みが終わると春。 今の時期はしぼりたての新酒が楽しめます。     ところで、酒造りに関するこんな噂、 きいたことありませんか？               【Ｒ-２】  石手川沿いの造り酒屋、栄光酒造。 冬から続いた仕込みがようやく終わり。 酒造りの職人、「蔵人」とともに作り上げた日本酒が 酒蔵に春を告げます。  ところでみなさん、 酒造りに関するこんな噂、聞いたことありませんか？   杜氏は、納豆を食べてはいけない。   杜氏の手は、白い。   いかがでしょうか。 真偽のほどは皆さんの目でお確かめください。  いえいえ、十分お綺麗です！ 蔵人の皆さんが作った様々なお酒は、 栄光酒造の直売所でも販売していますよ！     【Ｒ-３】  食パン一本で勝負！ ぷるっぷるの食パン登場！                        【Ｒ-４】ナレ無し　　インサート？   【Ｒ-５】 石手寺の目の前。去年９月にオープンした「パン屋アシヤ」 商品はこの食パンのみ！食パン一本で勝負する専門店です。 オープン当時は…   オープン初日からこの大行列！ 一度に何本も購入するお客さんも多いアシヤの食パン。 人気の秘密はこの生地。   まるでクリームのようになめらかな特製生地。 オーブンで焼くことおよそ３０分。 ぷるっぷるの食パンに仕上がります。   一日７００本以上売れるんですから、オーブンは大忙し！ プルプルもちもちのプレミアム食パンです。   もっちもちのプレミア食パン。お試しあれ。    【Ｒ-６】 石手川の河川敷でビーチバレー！ 